In [ ]:
%pip install PyPDF2
import PyPDF2

"""
Extracts text from PDF
Parameters:
    pdf_path (string): file path to pdf in directory tree
Returns a string of the pdf's contents
"""
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
        return text

pdf_path = "/content/EU AI Act.pdf"
document_text = extract_text_from_pdf(pdf_path)
print(document_text)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.8 MB/s eta 0:00:00
BRIEFING  
EU Legislation in Progress  
 
EPRS | European Parliamentary Research  Service  
Author: Tambiama  Madiega  
Members' Research Service 
PE 698.792  –  March 2024  EN 
Artificial intelligence act  
OVERVIEW  
European Union lawmakers reached a political agreement on the draft artificial intelligence (AI) act 
in December 2023. Proposed by the European Commission in April  2021, t he draft AI act, the first 
binding  worldwide  horizontal regulation  on AI, 
sets a common framework for the use and supply of 
AI systems in the EU. It offers  a classification for AI sy stems with different requirements and 
obligations tailored on a ' risk-based approach '. Some AI systems presenting 'unacceptable ' risks are 
prohibited. A wide range of 'high -risk' AI systems that can have a detrimental impact on people' s 
health, safety or on their fundamental rights  are authorised, but subject to a set of requi

In [ ]:
!pip install llama-index
!pip install pymilvus
import os
import numpy as np
from openai import AzureOpenAI
import textwrap
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.7 MB/s eta 0:00:00


In [ ]:
# Connect to Azure Openai
os.environ["AZURE_OPENAI_API_KEY"] = "4b3ea2d5074c4059ad670e016e83b853"
client = AzureOpenAI(
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = "2024-02-01",
    azure_endpoint = "https://dxc-eu-ai-act-chatbot.openai.azure.com/"
)

# model: gpt 35 turbo
# model version: 0301
deployment_name = "DXC"

# Send a test completion call to generate an answer
prompt = "What is the EU AI Act?"
response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 100,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)

print("User prompt: " + prompt)
print("Response: " + response.choices[0].text)

User prompt: What is the EU AI Act?
Response:  

The EU AI Act is a proposed set of regulations that would govern the development and use of artificial intelligence (AI) systems within the European Union (EU). The act is part of the EU's broader Digital Single Market strategy, which aims to create a single market for digital goods and services across the EU.

The EU AI Act would apply to a wide range of AI systems, including those used in healthcare, transportation, and finance. It would require developers to ensure that their systems are transparent, explain


In [ ]:
# Connect to Milvus
ENDPOINT = "https://in03-6a0d6245b69d362.serverless.gcp-us-west1.cloud.zilliz.com"
TOKEN = "c96536849f835f69648e8e1586f3e09794e9d9c63c2375d55bb62aa5a7b031bba566b53d6afd4410bb85f4aeea14f1406b947fe5"
connections.disconnect(alias="default")
connections.connect(
   uri = ENDPOINT,
   token = TOKEN)

# Define schema for Milvus collection
fields = [
    FieldSchema(name = "id", dtype = DataType.INT64, is_primary = True),
    FieldSchema(name = "embedding", dtype = DataType.FLOAT_VECTOR, dim = 1536),
    FieldSchema(name = "text", dtype = DataType.VARCHAR, max_length = 65535)
]

schema = CollectionSchema(fields, "DXC-EU-AI-Bot")
collection = Collection("eu_ai_act9", schema)

"""
  Generates embeddings using Azure OpenAI
  Parameters:
      text (string): section of text to create embeddings for
  Returns embedding of the text, vector of floats
"""
def generate_embedding(text):
    response = client.embeddings.create(
        input = text,
        model = "DXC-embedding"
    )
    return response.data[0].embedding

# Split the text into smaller chunks (vector of strings with max length of 5000 characters)
chunk_size = 1250
chunks = textwrap.wrap(document_text, chunk_size)
print("Example chunk: " + chunks[0])



Example chunk: BRIEFING   EU Legislation in Progress     EPRS | European Parliamentary Research  Service   Author: Tambiama  Madiega   Members' Research Service  PE 698.792  –  March 2024  EN  Artificial intelligence act   OVERVIEW   European Union lawmakers reached a political agreement on the draft artificial intelligence (AI) act  in December 2023. Proposed by the European Commission in April  2021, t he draft AI act, the first  binding  worldwide  horizontal regulation  on AI,  sets a common framework for the use and supply of  AI systems in the EU. It offers  a classification for AI sy stems with different requirements and  obligations tailored on a ' risk-based approach '. Some AI systems presenting 'unacceptable ' risks are  prohibited. A wide range of 'high -risk' AI systems that can have a detrimental impact on people' s  health, safety or on their fundamental rights  are authorised, but subject to a set of requirements and  obligations to gain access to the EU market. AI syst

In [ ]:
# Prepare data for insertion into Milvus
embeddings = np.array([generate_embedding(chunk) for chunk in chunks], dtype = np.float32)
ids = list(range(len(embeddings)))
data = [ids, embeddings, chunks]

# Insert into Milvus
collection.insert(data)
collection.flush()
print("Embeddings inserted into Milvus")

Embeddings inserted into Milvus


In [ ]:
index_params = {
    "index_type": "IVF_FLAT",
    "params": {"nlist": 128},
    "metric_type": "L2"
}

# Create an index on the embedding field in Milvus
collection.create_index(field_name = "embedding", index_params = index_params)

"""
  Perform a similarity search based on user query
  Parameters:
      query_embeddings (numpy array): user's query embedding
      top_k (int): max number of similar text chunks to return
  Returns list (strings) of similar text chunks from pdf
"""
def query_milvus(query_embedding, top_k = 3):
    collection.load()
    # if not collection.has_index():
    #     collection.create_index("embedding", index_params)

    results = collection.search(# Gathers list of metadata of the top sources
        data = query_embedding,
        anns_field = "embedding",
        param = {"metric_type": "L2", "params": {"nprobe": 15}},
        limit = top_k,
        output_fields=["text"]
    )
    for source in results:
      print(source)
    retrieved_chunks = [hit.entity.get("text") for hit in results[0]]
    return retrieved_chunks

In [ ]:
# Test query
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += " A client in the banking industry has approached our tech consulting company with a proposal for an AI-driven chatbot to assist customers with account management, including funds transfers, and fraud detection. How much risk does this financial services project have according to the EU AI Act? Please provide quotes and citations from the document."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.24225597083568573, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# Implementing a llm judge
llm_judging = f"""
You are an expert on the EU AI Act. The following is a response to a user query.
Please evaluate its correctness, relevance to the query, and completeness.

Query: {user_query}
Response: {response.choices[0].text}

Your evaluation should assign a score from 1 to 10, with 10 being perfect,
and provide a brief explanation for the score.
"""

judge_results = client.completions.create(
    model = deployment_name,
    prompt = llm_judging,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)

print("\nResponse: " + judge_results.choices[0].text)


Response: """
# I would give this response a score of 9. It is very informative and provides a clear answer to the query. The response cites the relevant articles from the EU AI Act and explains how they apply to the chatbot project proposed by the banking client. The only thing missing is a more detailed explanation of the conformity assessment process and what it entails. Otherwise, the response is very complete and relevant to the query. <|im_end|>


In [ ]:
# Prompting to use response critque to revise previous answer
revise_answer = f"""
You are an expert on the EU AI Act.
Give a new response better than the initial response by referencing the following context:{context}

Initial Response: {response.choices[0].text}
Feedback: {judge_results.choices[0].text}\n\n

{user_query}
"""

revise_results = client.completions.create(
    model = deployment_name,
    prompt = llm_judging,
    temperature = 1,
    max_tokens = 500,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)

print("\nResponse: " + revise_results.choices[0].text)


Response: """

# Score: 9
# The response provides a clear and concise answer to the query, including quotes and citations from the EU AI Act. The only improvement would be to provide more specific information on the risks posed by the chatbot project, rather than simply stating that it is a high-risk AI system. For example, the response could explain how the chatbot's use of customer data for funds transfers and fraud detection could pose risks to privacy and security. Overall, the response is highly relevant to the query and provides comprehensive information on the EU AI Act's requirements for high-risk AI systems. The only minor issue is that the last sentence seems incomplete and could benefit from further elaboration.
<|im_end|>


In [ ]:
# test query #2
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += " A client has a fintech startup and has approached our tech consulting company with a proposal to use AI to assess creditworthiness for loans by analyzing user data, including social media activity and purchasing history. The startup wants to ensure that they comply with the EU AI Act. Please provide quotes and citations from the document."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question based on the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 500,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.25235551595687866, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# Implementing a llm judge
llm_judging = f"""
You are an expert on the EU AI Act. The following is a response to a user query.
Please evaluate its correctness, relevance to the query, and completeness.

Query: {user_query}
Response: {response.choices[0].text}
Context: {context}

Your evaluation should assign a score from 1 to 10, with 10 being perfect,
and provide a brief explanation for the score.
"""

judge_results = client.completions.create(
    model = deployment_name,
    prompt = llm_judging,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)

print("\nResponse: " + judge_results.choices[0].text)


Response: """

score = 3
explanation = "The response does not answer the user query. Instead, it poses many questions that are not relevant to the user's needs. The response should have provided specific quotes and citations from the EU AI Act that address the use of AI in creditworthiness assessments. Score: 3/10."
print(explanation)<|im_sep|>


In [ ]:
# Prompting to use response critque to revise previous answer
revise_answer = f"""
You are an expert on the EU AI Act.
Give a new response better than the initial response by referencing the following context:{context}

Initial Response: {response.choices[0].text}
Feedback: {judge_results.choices[0].text}
Context: {context}\n\n

{user_query}
"""

revise_results = client.completions.create(
    model = deployment_name,
    prompt = llm_judging,
    temperature = 1,
    max_tokens = 500,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)

print("\nResponse: " + revise_results.choices[0].text)


Response: """

score = 1
explanation = "The response does not answer the user query and instead provides a list of questions and topics related to the EU AI Act. It does not provide any quotes or citations from the document, nor does it address the specific use case of the fintech startup. The response is not relevant, complete, or correct."
print(f"Score: {score}; Explanation: {explanation}")<|im_sep|>


In [ ]:
# 1
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A company develops an AI system for diagnosing diseases using medical imaging. Evaluate how the EU AI Act would classify this AI system. What specific requirements would the company need to meet to gain market access in the EU? (Give a paragraph max)"
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.22884008288383484, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 2
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A company produces smart home devices that use AI for energy management and security. Discuss the categorization of these smart home devices under the EU AI Act. What compliance measures should the company consider? (Give a paragraph max)"
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 150,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.25549888610839844, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 3
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A company develops an AI tool to screen job applicants and rank them based on qualifications.Evaluate the ethical and regulatory considerations under the EU AI Act for this recruitment tool. What transparency requirements should be implemented? (Give a paragraph max)"
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 200,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.23626196384429932, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 4
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A bank uses AI algorithms to detect fraudulent transactions in real-time.Examine how the EU AI Act’s provisions for high-risk AI systems would affect the bank’s fraud detection AI. What measures must be taken to comply? (Give a paragraph max)"
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.23696768283843994, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 5
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "An online retailer implements an AI chatbot to handle customer inquiries and complaints. Identify the requirements of the EU AI Act that would apply to this AI system. How can the company ensure compliance? (Give a paragraph max)"
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.22680127620697021, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 6
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "An educational platform uses AI to personalize learning paths for students. What risks related to data privacy and student safety might this AI system present? How would these risks be evaluated under the EU AI Act? (Give a paragraph max)"
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.24222332239151, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage pro

In [ ]:
# 7
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A city employs facial recognition technology for surveillance in public areas. What privacy and ethical risks are associated with the use of facial recognition? How would the EU AI Act evaluate these risks? (Give a paragraph max)"
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 35, distance: 0.24168550968170166, entity: {'text': '2021.   9  See M. Ebers and others, above.   10  See V. Galaz and others, Artificial intelligence, systemic risks, and sustainability , Vol 67, Technology in Society , 2021.   EPRS | European Parliamentary Research Service   12   11  For an overview, see T. Madiega and H. Mildebrath, Regulating facial recognition in the EU, 2021.   12  The act applies to private organisations as well as to public authorities.   13  The Annex refers  to AI systems used in areas of  critical infrastructures (e.g.  road traffic ), education and vocational  training , employment  worker management and access to self -employment , access to essential private and public  services  and benefits  (e.g., creditworthiness evaluation ), law enforcement, border control, administration of justice  and democratic processes , biometric identification, categorisation and emotion recognit ion systems (outside the  prohibited categories) .  14  An AI system will

In [ ]:
# 8
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A healthcare startup develops an AI tool to predict disease outbreaks based on environmental data. Evaluate the risks associated with false positives and misinterpretation of data. What classifications under the EU AI Act would be relevant? (Give a paragraph max)"
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.23612521588802338, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 9
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A police department uses an AI system to predict crime hotspots based on historical data. Identify the types of risks associated with this predictive policing AI. What are the potential ethical and societal implications under the EU AI Act? (Give a paragraph max)"
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.23359991610050201, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 10
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A tech company creates an AI voice recognition system for customer service applications. Analyze the risks related to data security and user consent in the use of voice data. What requirements must the company fulfill? (Give a paragraph max)"
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.23771032691001892, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 11
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A client wants to use an AI algorithm trained on customer data and trends to create targeted sales and marketing campaigns for customers to drive revenue and customer loyalty as well as spending. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.22368264198303223, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 12
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A bank approaches you with a proposal to use AI to determine whether a client is suitable to be approved for a loan or not based on financial history and racial background. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act. "
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.2176882028579712, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage p

In [ ]:
# 13
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "An airport wants to establish a system that determines the level of risk an individual may be assoicated with so they can increase checks on high risk individuals. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.2238108217716217, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage p

In [ ]:
# 14
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A company wants to implement a new AI system that can be used to decide who should be promoted and how much they should be paid. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.2197062373161316, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage p

In [ ]:
# 15
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A company wants to use an AI system for cybersecurity to identify potential threats to systems and employees that may be easily social engineered. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.20720139145851135, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 16
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A company wants to implement an AI system to manage inventory and distribution for supply chain operations. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.21998894214630127, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 17
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A school district plans to develop and implement an AI system to assist with budget allocation. This application will decide how much funding each school in the district received based on student profiles including performance and race, previous funding history, and location among other factors. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.21299906075000763, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 18
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A fitness center wants to implement AI technology into their gym equipment to help users better analyze their data and receive insights from their exercise sessions. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.23451580107212067, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 19
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A tax agency wants to design and implement a new accounting system that uses AI technology to help clients quickly fill out tax documents and perform financial audits. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.2235727608203888, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage p

In [ ]:
# 20
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A ride share company wants to integrate AI into their current system to better match drivers with riders based on personal information in addition to route details. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.2150665521621704, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage p

In [ ]:
# 21
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A student wants to make an app that provides users with AI aided summaries of financial news and updates on market trends. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.22359025478363037, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 22
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A bank deploys an AI chatbot to answer general FAQs about branch hours, account types, and ATM locations. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.22708766162395477, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 23
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A bank wants to make a multilingual chatbot that can translate customer inquiries and responses to support non-native speakers in general banking queries (e.g., branch info, contact details). According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.22267088294029236, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 24
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A business wants to make an AI that can analyze public social media content to gauge general customer sentiment about their brand. It aggregates data to provide an overall view, without individual user data. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.21955469250679016, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 25
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A business makes an AI tool that checks if a signature field on scanned forms is filled in before submission without verifying or matching the signature. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.24012023210525513, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 26
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A business makes an AI tool that extracts handwritten text (e.g., name, date) from deposit slips to pre-fill digital forms, improving speed and reducing errors. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.22769862413406372, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 27
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A social media company makes an application that uses facial landmark detection to help users create avatars based on their facial features in video games or virtual spaces. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.2431918978691101, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage p

In [ ]:
# 28
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A group of scientists want to make an AI aplication that aids in counting cells or bacteria on microscope slides for laboratory studies, improving speed and accuracy in data collection. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.24275043606758118, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 29
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A group of meteorologists want to create an AI that can detect cloud formations or weather patterns in satellite images to aid in meteorological studies without identifying individual locations or people. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.23200929164886475, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 30
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "Environmental researchers want to use AI to identify microplastic particles in water samples under a microscope, allowing them to quantify pollution levels. According to the EU AI Act, what level of risk is associated with this project  and how can the risks be mitigated? Please provide direct citations from the EU AI Act."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

['id: 31, distance: 0.24621540307998657, entity: {\'text\': "States  circumven t the rules enshrined in the AI act.22 Kutterer argues the AI act \'s implementation will require  a robust taxonomy setting out the correlation of risk classification and model capabilities and asses sing  the  developments of open sources models.23 Helberger and others call for  the AI act to be complemented by  an  additional set of exercisable rights to protect citizens from AI-generated harm, with additional legislation to Artificial intelligence act   11 control the potential environmental impact of training AI models and protect worker\'s rights and to define  further a set of requirements that research organisations must comply with to benefit from the research  exemption.24 Also, some argue  that the AI act d oes not go far enough in preventing and/or mitigating the  specific risks associated with chatbots. T imely standardisation will be key to ensur ing adequate  implementation of the AI act, for 

In [ ]:
# 31
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "How does the EU AI Act address AI systems that are categorized as “unacceptable risk,” and what are some examples of these systems?"
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.22568431496620178, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 32
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "What penalties or enforcement actions does the EU AI Act outline for non-compliance with risk-related regulations?"
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.24659527838230133, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 33
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "How does the EU AI Act guide the continuous monitoring and reassessment of an AI system’s risk level?"
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.2430921196937561, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage p

In [ ]:
# 34
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "An AI-driven hiring platform that screens and ranks job candidates based on their resumes and online presence."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.2949616312980652, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage p

In [ ]:
# 35
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "A medical diagnostic AI tool that assists doctors by analyzing patient data to suggest diagnoses. "
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.29126301407814026, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 36
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "An AI model for an autonomous driving system that controls vehicle navigation and decision-making on public roads."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.27751436829566956, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage 

In [ ]:
# 37
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "An AI-based personalized learning platform that adjusts educational content based on students' progress and learning style."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

['id: 20, distance: 0.30115967988967896, entity: {\'text\': "approaches, and the Commission has been tasked to develop guidelines  on its application.     The act also contains a definition of g eneral purpose  artificial intelligence  (GPAI) models  \'that  are trained with a large amount of data using self -supervision at scale \', that display \' significant  generality \' and are  \'capable to competently perform a wide range of distinct tasks\' and \'can be  integrated into a variety of downstream systems or applications\' . Furthermore, t he AI act defines  general -purpose AI systems  as systems based on a GPAI model, which have the capability to serve  a variety of purposes, both for direct use as well as for integration in other AI systems.    Scope of application    The AI act  applies  primarily to providers and deployers putting AI systems and GPAI models into  service or placing  on the EU market and who have their place of establishment or who are located  in the EU, as w

In [ ]:
# 38
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "An AI that analyzes historical crime data to predict potential future crime hotspots, guiding police resource allocation."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

['id: 34, distance: 0.3040567636489868, entity: {\'text\': "of crime, including missing children ; (ii) to  prevent a specific, substantial and imminent threat to the life or physical safety of persons or of a terrorist attack ;  and (iii)  for the detection, localisation, identification or prosecution of a per petrator or individual suspected of a  criminal offence referred to in the European Arrest Warrant Framework Decision .  6  This section aims to provide a flavour of the de bate and is not intended to be an exhaustive account of all different  views on the proposal. Additional information can be found in publications listed under \'supporting analysis\'.   7  For an in -depth analysis of the proposals and recommendations for amendm ents see N. Smuha et al. , How the EU  can achieve legally trustworthy AI: A response to the European Commission \'s proposal for an a rtificial intelligence  act, Elsevier, August 2021; M. Ebers, and others, The European Commission’s proposal for an 

In [ ]:
# 39
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "An AI system used by law enforcement to identify individuals in real-time through facial recognition in public places."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 35, distance: 0.2734036445617676, entity: {'text': '2021.   9  See M. Ebers and others, above.   10  See V. Galaz and others, Artificial intelligence, systemic risks, and sustainability , Vol 67, Technology in Society , 2021.   EPRS | European Parliamentary Research Service   12   11  For an overview, see T. Madiega and H. Mildebrath, Regulating facial recognition in the EU, 2021.   12  The act applies to private organisations as well as to public authorities.   13  The Annex refers  to AI systems used in areas of  critical infrastructures (e.g.  road traffic ), education and vocational  training , employment  worker management and access to self -employment , access to essential private and public  services  and benefits  (e.g., creditworthiness evaluation ), law enforcement, border control, administration of justice  and democratic processes , biometric identification, categorisation and emotion recognit ion systems (outside the  prohibited categories) .  14  An AI system will 

In [ ]:
# 40
user_query = "We are a technology consulting company with global clients. It is important for our company and the clients of our company to stay within regulations of the strictest AI act, the EU AI Act."
user_query += "An AI system deployed to detect and remove harmful or illegal content on social media platforms."
query_embedding = np.array([generate_embedding(user_query)], dtype = np.float32)
relevant_text_chunks = query_milvus(query_embedding, 3)
context = " ".join(relevant_text_chunks)
prompt = f"Answer the question by referencing the following context: {context}\n\n{user_query}"

# Print metadata of the top retrieved text chunks from Milvus
print(user_query + "\n")
print("Top Sources:")
num = 1
for chunk in relevant_text_chunks:
    print("   " + str(num) + ". " + chunk[:200])
    num += 1

response = client.completions.create(
    model = deployment_name,
    prompt = prompt,
    temperature = 1,
    max_tokens = 250,
    top_p = 0.5,
    frequency_penalty = 0,
    presence_penalty = 0,
    stop = None
)
print("\nResponse: " + response.choices[0].text)

["id: 8, distance: 0.2833715081214905, entity: {'text': 'requirements  particularly on risk  management, testing, technical robustness, data training and data governance, transparency,   human oversight, and cybersecurity before being placed on the market or put  into service. AI  systems that conform to new harmonised EU standards would benefit from a presumption of  conformity with the draft AI act requirements.  T hird, AI systems presentin g limited risk , such as systems that interacts with humans (i.e. chatbots),  emotion recognition systems, biometric categorisation systems, and AI systems that generate or  manipulate image, audio or video content (i.e. deepfakes), would be subject to a limi ted set of  transparency obligations.   Finally,  all other AI systems presenting only low or minimal risk  could be developed and used in  the EU without conforming to any additional legal obligations. However, the proposed AI act  envisage d the creation of codes of conduct  to encourage p